In [21]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
import os

In [2]:
local=True
if local:
    spark=SparkSession.builder.master("local[4]").appName("filter_Ais").getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("filter_Ais") \
                      .config("spark.kubernetes.container.image","inseefrlab/jupyter-datascience:master") \
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "4") \
                      .config("spark.executor.memory","8g") \
                      .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1') \
                      .getOrCreate()      

2022-11-09 12:28:02,590 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
ais_file_path="s3a://projet-hackathon-un-2022/AIS/ais_azov_black_20220001_20220007.parquet"
his_file_path="s3a://projet-hackathon-un-2022/IHS/ship_data.parquet"

In [7]:
df_ais=spark.read.parquet(ais_file_path)

In [8]:
df_ais.show(n=1, truncate=False, vertical=True)

-RECORD 0-----------------------------
 hex_resolution | 8                   
 longitude      | 31.40833333         
 mmsi           | 272157700           
 destination    | MYKOLAIV            
 eeid           | 5191743282127358980 
 H3_int_index_8 | 613021963599740927  
 latitude       | 47.52666667         
 dt_insert_utc  | 2022-01-01 05:58:49 
 polygon_name   | Polygon             
only showing top 1 row



In [9]:
df_his=spark.read.parquet(his_file_path)

In [11]:
df_his.show(n=1,truncate=False,vertical=True)

2022-11-09 12:41:21,572 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 LRIMOShipNo                               | 1000019                                                                                                                                                                            
 StatCode5                                 | X11A2YP                                                                                                                                                                            
 AlterationsDescriptiveNarrative           | null                                                                                                                                                                               
 PropulsionTypeCode                        | DD                                                     

In [12]:
df_full=df_ais.join(df_his,df_ais.mmsi==df_his.MaritimeMobileServiceIdentityMMSINumber,"inner")

In [13]:
df_full.show(n=1,truncate=False,vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 hex_resolution                            | 8                                                                                                                                                                                                         
 longitude                                 | 27.45579667                                                                                                                                                                                               
 mmsi                                      | 207277000                                                                                                                                                                                                 
 destina

# scenario 1

We want to know the percentage of the ship number of a certain day of the whole data set. 
- get the ship number of the whole data set
- get the ship number of the given day
- get the percentage

In [19]:
ship_id_col="mmsi"
date_col="dt_insert_utc"

total_ship_number=df_ais.select(ship_id_col).distinct().count()
print(total_ship_number)

3253


In [44]:
target_date="2022-01-04"

ship_number_of_day=df_ais.withColumn("date",F.to_date(date_col)).filter(F.col("date") == F.lit(target_date)).select(ship_id_col).distinct().count()
print(ship_number_of_day)

2328


In [45]:
number_percentage=(ship_number_of_day/total_ship_number)*100
print(number_percentage)

71.56470949892407


In [54]:
# df_all should be a data frame return by the af.get_ais() with certain fi

def get_ship_nb_percentage_of_day(df_all,target_date):
    ship_id_col="mmsi"
    date_col="dt_insert_utc"
    total_ship_number=df_all.select(ship_id_col).distinct().count()
    ship_number_of_day=df_all.withColumn("date",F.to_date(date_col)).filter(F.col("date") == F.lit(target_date)).select(ship_id_col).distinct().count()
    return (ship_number_of_day/total_ship_number)*100

In [55]:
res=get_ship_nb_percentage_of_day(df_ais,target_date)
print(res)

71.56470949892407


# scenario 2

We want to know the percentage of the ship tonage of a certain day of the whole data set. 
- get the distinct ship of the whole data set, and sum their tonage 
- get the distinct ship of the given day, and sum their tonage
- get the percentage

In [59]:
all_ship_mmsi=df_ais.select(ship_id_col).distinct().toPandas()['mmsi'].tolist()

In [ ]:
print(all_ship_mmsi)

In [61]:
all_ship_tonage=df_his.filter(F.col("MaritimeMobileServiceIdentityMMSINumber").isin(all_ship_mmsi)).agg(F.sum("GrossTonnage")).collect()[0][0]
print(all_ship_tonage)

20633751


In [62]:
ship_mmsi_of_day=df_ais.withColumn("date",F.to_date(date_col)).filter(F.col("date") == F.lit(target_date)).select(ship_id_col).distinct().toPandas()['mmsi'].tolist()

In [ ]:
print(ship_mmsi_of_day)

In [64]:
ship_tonage_of_day=df_his.filter(F.col("MaritimeMobileServiceIdentityMMSINumber").isin(ship_mmsi_of_day)).agg(F.sum("GrossTonnage")).collect()[0][0]
print(ship_tonage_of_day)

14822274


In [65]:
tonage_percentage=(ship_tonage_of_day/all_ship_tonage)*100
print(tonage_percentage)

71.83509193263018


In [56]:
def get_ship_tonage_percentage_of_day(df_all,df_his,target_date):
    ship_id_col="mmsi"
    date_col="dt_insert_utc"
    all_ship_mmsi=df_all.select(ship_id_col).distinct().toPandas()['mmsi'].tolist()
    all_ship_tonage=df_his.filter(F.col("MaritimeMobileServiceIdentityMMSINumber").isin(all_ship_mmsi)).agg(F.sum("GrossTonnage")).collect()[0][0]
    ship_mmsi_of_day=df_all.withColumn("date",F.to_date(date_col)).filter(F.col("date") == F.lit(target_date)).select(ship_id_col).distinct().toPandas()['mmsi'].tolist()
    ship_tonage_of_day=df_his.filter(F.col("MaritimeMobileServiceIdentityMMSINumber").isin(ship_mmsi_of_day)).agg(F.sum("GrossTonnage")).collect()[0][0]
    return (ship_tonage_of_day/all_ship_tonage)*100

In [58]:
res=get_ship_tonage_percentage_of_day(df_ais,df_his,target_date)
print(res)

71.83509193263018


# scenario 3

We want to know the percentage of the ship number of a certain area (a given polygon) of the whole data set. 
- get the ship number of the whole data set
- get the ship number of the given zone (We filter a zone by their polygon_name)
- get the percentage

In [ ]:
# Set coordinates of the selected polygons in geojson format
# https://boundingbox.klokantech.com/

azov_black_geoj = [[43.3308500839,39.9913666442],[26.1506878922,41.33737686],[27.1872912828,48.4341912681],[44.3674534746,47.2431326615],[43.3308500839,39.9913666442]]

polygon = {
        "type": "Polygon",
        "coordinates": [bb]
    }

azov_black_df = af.polygon_to_hex_df([("polygon_azov_black", polygon)])

# Filter boats that were at least 1 time in our polygon

start_date = datetime.fromisoformat("2022-01-01")
end_date = datetime.fromisoformat("2022-06-30")
columns = ["mmsi", "latitude", "longitude", "eeid", "dt_insert_utc", "destination"]



In [ ]:
ship_id_col="mmsi"

# get world wide AIS
df_all = af.get_ais(spark,
                            start_date, 
                            end_date = end_date,
                            columns = columns
                           )
total_ship_number=df_all.select(ship_id_col).distinct().count()

In [ ]:
df_azov_black = af.get_ais(spark,
                            start_date, 
                            end_date = end_date,
                            columns = columns,
                            polygon_hex_df = azov_black_df
                           )

azov_black_ship_number=df_azov_black.select(ship_id_col).distinct().count()

In [ ]:
percentage=(azov_black_ship_number/total_ship_number)*100

# scenario 4

We want to know the percentage of the ship tonage of a certain day of the whole data set. 
- get the distinct ship of the whole data set, and sum their tonage 
- get the distinct ship of the given day, and sum their tonage
- get the percentage

In [ ]:
all_ship_mmsi=df_all.select(ship_id_col).distinct().toPandas()['mmsi'].tolist()
all_ship_tonage=df_his.filter(F.col("MaritimeMobileServiceIdentityMMSINumber").isin(all_ship_mmsi)).agg(F.sum("GrossTonnage")).collect()[0][0]
print(all_ship_tonage)

In [ ]:
ship_mmsi_of_azov_black=df_ais.select(ship_id_col).distinct().toPandas()['mmsi'].tolist()
ship_tonage_of_azov=df_his.filter(F.col("MaritimeMobileServiceIdentityMMSINumber").isin(all_ship_mmsi)).agg(F.sum("GrossTonnage")).collect()[0][0]
print(ship_tonage_of_azov)

In [ ]:
tonage_percetage=(ship_tonage_of_azov/all_ship_tonage)*100

# helper function get mmsi list by day

We want to get the mmsi list of a dataset day by day

In [68]:
def get_distinct_mmsi_by_day(df_all):
    return df_all.withColumn("date",F.to_date(date_col)).groupBy("date").agg(F.collect_set("mmsi").alias("distinct_mmsi_by_day")).orderBy("date")

In [69]:
df_mmsi_by_day=get_distinct_mmsi_by_day(df_ais)
df_mmsi_by_day.show()

+----------+--------------------+
|      date|distinct_mmsi_by_day|
+----------+--------------------+
|2022-01-01|[264163452, 27333...|
|2022-01-02|[273332380, 26416...|
|2022-01-03|[264163452, 27333...|
|2022-01-04|[264163452, 27333...|
|2022-01-05|[271048605, 27333...|
|2022-01-06|[273332380, 21512...|
|2022-01-07|[273332380, 27104...|
+----------+--------------------+

